In [16]:
import os
import joblib
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

os.makedirs("models", exist_ok=True)
os.makedirs("results", exist_ok=True)

iris = load_iris()
X = iris.data
y = iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

def evaluate_and_save(model, model_name):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, average="weighted")
    rec = recall_score(y_test, y_pred, average="weighted")
    f1 = f1_score(y_test, y_pred, average="weighted")

    model_path = os.path.join("models", f"{model_name}.pkl")
    joblib.dump(model, model_path)

    print(f"\n{model_name} Results (saved at {model_path}):")
    print(f"Accuracy: {acc:.3f}, Precision: {prec:.3f}, Recall: {rec:.3f}, F1-score: {f1:.3f}")

    return [model_name, acc, prec, rec, f1]

log_reg = LogisticRegression(max_iter=200)
log_reg_results = evaluate_and_save(log_reg, "logistic_regression")

rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf_results = evaluate_and_save(rf, "random_forest")

svm = SVC(kernel="linear", probability=True, random_state=42)
svm_results = evaluate_and_save(svm, "svm")

results = [log_reg_results, rf_results, svm_results]
df_results = pd.DataFrame(results, columns=["Model", "Accuracy", "Precision", "Recall", "F1-score"])
df_results.to_csv("results/model_comparison.csv", index=False)

print("\nComparison table saved to results/model_comparison.csv")
print(df_results)


logistic_regression Results (saved at models\logistic_regression.pkl):
Accuracy: 0.933, Precision: 0.933, Recall: 0.933, F1-score: 0.933

random_forest Results (saved at models\random_forest.pkl):
Accuracy: 0.900, Precision: 0.902, Recall: 0.900, F1-score: 0.900

svm Results (saved at models\svm.pkl):
Accuracy: 1.000, Precision: 1.000, Recall: 1.000, F1-score: 1.000

Comparison table saved to results/model_comparison.csv
                 Model  Accuracy  Precision    Recall  F1-score
0  logistic_regression  0.933333   0.933333  0.933333  0.933333
1        random_forest  0.900000   0.902357  0.900000  0.899749
2                  svm  1.000000   1.000000  1.000000  1.000000
